### Importing required libraries

In [1]:
#Importing required libraries
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from datetime import date
import pandas as pd

### Starting webdriver connection in Chrome

In [15]:
driver1 = webdriver.Chrome(executable_path = "chromedriver.exe")
driver1.set_window_size(720,1280)

### Class with methods to create dataframe of data gathered from site and to remove improper data and converting to numerical values

In [ ]:
class Scrapedata:
    #Method to create dataframe of data gathered from site
    def datastock(st_day,st_month,st_year,com_name):
        #Getting today's date until which to find the data
        today = date.today()

        #Ensuring that you get the correct date
        # while True:
        #     if(st_day in range(1,32)):
        #         break
        #     st_day = int(input("Enter starting date(correct day): "))
        # while True:
        #     if(st_month in range(1,13)):
        #         break
        #     st_month = int(input("Enter starting date(correct month): "))
        # while True:
        #     if(st_year in range(2008,today.year)):
        #         break
        #     st_year = int(input("Enter starting date(correct year): "))

        #Getting the lowest date you can enter
        d0 = date(2008,1,1)
        #Initial date code for getting url
        init_period = 1199145600
        #amount to be added for getting next day code
        day_gap = 86400

        #Getting the code for starting date
        d1 = date(st_year, st_month, st_day)
        delta1 = d1 - d0
        period1 = init_period + delta1.days * day_gap

        #Getting the code for ending date
        d2 = date(today.year, today.month, today.day)
        delta2 = d2 - d0
        period2 = init_period + delta2.days * day_gap

        #To get the number of days in this period
        delta = d2 - d1

        #com_name = input("Enter Name of Stock(with NS/BO): ")

        #driver1 = webdriver.Chrome(executable_path = "chromedriver.exe")
        #driver1.set_window_size(720,1280)

        #Opening the link
        driver1.get("https://finance.yahoo.com/quote/"+com_name+"/history?period1="+str(period1)+"&period2="+str(period2)+"&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true")

        #To scroll to end of the web page
        for k in range(0,delta.days, 100):
            try :
                ActionChains(driver1).move_to_element(driver1.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/table/tbody/tr["+str(k)+"]")).perform()
                continue
            except :
                driver1.execute_script("window.scrollTo(0, 200000);")

        #Getting the number of days for which data is there in the site
        a = driver1.find_element_by_xpath('/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div').find_elements_by_tag_name('tr')
        a = a[1:-1]
        
        #Creating empty lists
        s_date = []
        s_open = []
        s_high = []
        s_low = []
        s_close = []
        s_adjclose = []
        s_vol = []

        #Getting the data in lists 
        for i in range(1, len(a)+1):
            try:
                diff = driver1.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/table/tbody/tr["+str(i)+"]/td[2]").text
                if(diff.split(" ")[1] == "Dividend" or diff.split(" ")[1] == "Stock" or diff == "-"):
                    continue
            except:
                pass
            s_date.append(driver1.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/table/tbody/tr["+str(i)+"]/td[1]").text)
            s_open.append((driver1.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/table/tbody/tr["+str(i)+"]/td[2]").text).replace(",",""))
            s_high.append((driver1.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/table/tbody/tr["+str(i)+"]/td[3]").text).replace(",",""))
            s_low.append((driver1.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/table/tbody/tr["+str(i)+"]/td[4]").text).replace(",",""))
            s_close.append((driver1.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/table/tbody/tr["+str(i)+"]/td[5]").text).replace(",",""))
            s_adjclose.append((driver1.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/table/tbody/tr["+str(i)+"]/td[6]").text).replace(",",""))
            s_vol.append((driver1.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/table/tbody/tr["+str(i)+"]/td[7]").text).replace(",",""))
        
        #Reversing the lists to get dates in ascending order
        s_date.reverse()
        s_high.reverse()
        s_low.reverse()
        s_open.reverse()
        s_close.reverse()
        s_vol.reverse()
        s_adjclose.reverse()

        #Creating dictionary for these lists
        stock_data = {'Date' :s_date, 'High':s_high, 'Low': s_low,'Open': s_open,'Close': s_close, 'Volume': s_vol, 'Adj Close': s_adjclose}
        #Creating dataframe from dictionary
        sd_df = pd.DataFrame(stock_data)
        
        #Returning dataframe by removing improper values and converting to numeric values
        return Scrapedata.remove_improper_values_and_convert_to_numeric(sd_df)
        

    #Method for preprocessing the data
    def remove_improper_values_and_convert_to_numeric(a):
        #Removing improper values
        a = a[a['High'] != '-']
        a = a[a['Low'] != '-']
        a = a[a['Open'] != '-']
        a = a[a['Close'] != '-']
        a = a[a['Volume'] != '-']
        a = a[a['Adj Close'] != '-']
        #Converting to numeric values
        a['High'] = pd.to_numeric(a['High'])
        a['Low'] = pd.to_numeric(a['Low'])
        a['Open'] = pd.to_numeric(a['Open'])
        a['Close'] = pd.to_numeric(a['Close'])
        a['Volume'] = pd.to_numeric(a['Volume'])
        a['Adj Close'] = pd.to_numeric(a['Adj Close'])
        #Returning processed dataframe
        return a


### Lists for companies' codes and names

In [ ]:
com_list = ['VENKEYS.NS','TATASTLLP.NS','JKTYRE.NS','RAYMOND.NS','TATAMETALI.NS','TCS.NS','CTSH','INFY','HCLTECH.NS','WIPRO.NS','ONGC.NS','RAJESHEXPO.NS','ICICIBANK.NS','HDFCBANK.NS','LTI.NS','ASTRAL.NS','DEEPAKNTR.NS','PERSISTENT.NS','MINDTREE.NS','NAUKRI.NS','%5ENSEI']
com_name_list = ['VENKEYS','TATASTLLP','JKTYRE','RAYMOND','TATAMETALI','TCS','COGNIZANT','INFOSYS','HCL','WIPRO','ONGC','RAJESHEXPORTS','ICICI','HDFC','LTI','ASTRAL','DEEPAKNTR','PERSISTENT','MINDTREE','NAUKRI','NIFTY']

### Running the function defined above for each company in this list

In [ ]:
for i in range(len(com_list)):
    stock_data = Scrapedata.datastock(18,11,2018,com_list[i])
    #Exportingto company-name.csv
    stock_data.to_csv(com_name_list[i]+'.csv',index = False)

### Running function for MINDTREE. Since we need for different period, we do it seperately

In [ ]:
stock_data = Scrapedata.datastock(1,1,2012,"MINDTREE.NS")
#Exporting to MINDTREE.csv
stock_data.to_csv("MINDTREE-PREDICTION.csv",index = False)

### For NIFTY 50, We only need the "Date", "Open", "High", "Low", "Close" columns. So, We slightly re-define our class for specifically retrieving these columns only.

In [13]:
class Scrapedata_Nifty:
    #Method to create dataframe of data gathered from site
    def datastock(st_day,st_month,st_year,com_name):
        #Getting today's date until which to find the data
        today = date.today()

        #Ensuring that you get the correct date
        # while True:
        #     if(st_day in range(1,32)):
        #         break
        #     st_day = int(input("Enter starting date(correct day): "))
        # while True:
        #     if(st_month in range(1,13)):
        #         break
        #     st_month = int(input("Enter starting date(correct month): "))
        # while True:
        #     if(st_year in range(2008,today.year)):
        #         break
        #     st_year = int(input("Enter starting date(correct year): "))

        #Getting the lowest date you can enter
        d0 = date(2008,1,1)
        #Initial date code for getting url
        init_period = 1199145600
        #amount to be added for getting next day code
        day_gap = 86400

        #Getting the code for starting date
        d1 = date(st_year, st_month, st_day)
        delta1 = d1 - d0
        period1 = init_period + delta1.days * day_gap

        #Getting the code for ending date
        d2 = date(today.year, today.month, today.day)
        delta2 = d2 - d0
        period2 = init_period + delta2.days * day_gap

        #To get the number of days in this period
        delta = d2 - d1

        #com_name = input("Enter Name of Stock(with NS/BO): ")

        #driver1 = webdriver.Chrome(executable_path = "chromedriver.exe")
        #driver1.set_window_size(720,1280)

        #Opening the link
        driver1.get("https://finance.yahoo.com/quote/"+com_name+"/history?period1="+str(period1)+"&period2="+str(period2)+"&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true")

        #To scroll to end of the web page
        for k in range(0,delta.days, 100):
            try :
                ActionChains(driver1).move_to_element(driver1.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/table/tbody/tr["+str(k)+"]")).perform()
                continue
            except :
                driver1.execute_script("window.scrollTo(0, 200000);")

        #Getting the number of days for which data is there in the site
        a = driver1.find_element_by_xpath('/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div').find_elements_by_tag_name('tr')
        a = a[1:-1]
        
        #Creating empty lists
        s_date = []
        s_open = []
        s_high = []
        s_low = []
        s_close = []

        #Getting the data in lists 
        for i in range(1, len(a)+1):
            try:
                diff = driver1.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/table/tbody/tr["+str(i)+"]/td[2]").text
                if(diff.split(" ")[1] == "Dividend" or diff.split(" ")[1] == "Stock" or diff == "-"):
                    continue
            except:
                pass
            s_date.append(driver1.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/table/tbody/tr["+str(i)+"]/td[1]").text)
            s_open.append((driver1.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/table/tbody/tr["+str(i)+"]/td[2]").text).replace(",",""))
            s_high.append((driver1.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/table/tbody/tr["+str(i)+"]/td[3]").text).replace(",",""))
            s_low.append((driver1.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/table/tbody/tr["+str(i)+"]/td[4]").text).replace(",",""))
            s_close.append((driver1.find_element_by_xpath("/html/body/div[1]/div/div/div[1]/div/div[3]/div[1]/div/div[2]/div/div/section/div[2]/table/tbody/tr["+str(i)+"]/td[5]").text).replace(",",""))
        
        #Reversing the lists to get dates in ascending order
        s_date.reverse()
        s_high.reverse()
        s_low.reverse()
        s_open.reverse()
        s_close.reverse()

        #Creating dictionary for these lists
        stock_data = {'Date' :s_date, 'Open':s_open, 'High': s_high,'Low': s_low,'Close': s_close}
        #Creating dataframe from dictionary
        sd_df = pd.DataFrame(stock_data)
        
        #Returning dataframe by removing improper values and converting to numeric values
        return Scrapedata_Nifty.remove_improper_values_and_convert_to_numeric(sd_df)
        

    #Method for preprocessing the data
    def remove_improper_values_and_convert_to_numeric(a):
        #Removing improper values
        a = a[a['High'] != '-']
        a = a[a['Low'] != '-']
        a = a[a['Open'] != '-']
        a = a[a['Close'] != '-']
        #Converting to numeric values
        a['High'] = pd.to_numeric(a['High'])
        a['Low'] = pd.to_numeric(a['Low'])
        a['Open'] = pd.to_numeric(a['Open'])
        a['Close'] = pd.to_numeric(a['Close'])
        #Returning processed dataframe
        return a

### Running function for NIFTY 50. Since we need for different period, we do it seperately

In [16]:
stock_data = Scrapedata_Nifty.datastock(1,1,2011,"%5ENSEI")
#Exporting to NIFTY 50_Data.csv
stock_data.to_csv("NIFTY 50_Data.csv",index = False)

### Closing webdriver connection

In [ ]:
driver1.close()